# Data Mining in Physics - Presentation 3. - DAGUR 5.4: Assessing predictive accuracy

Main goal examined in this section is how to minimize standard errors and confidence intervals of a regression model and thus increase its accuracy.

In [ ]:
library(microbenchmark)
library(graphics)
library(ggplot2)

In [ ]:
options(jupyter.plot_scale=1.4)

In [ ]:
library(lattice)
library(DAAG)

## 1. Linear fit the `DAAG::houseprices` dataset

In [ ]:
# Fit linear model on `houseprices` dataset
houseprices.lm = lm(sale.price ~ area, data=houseprices)
# Print summary
summary(houseprices.lm)

In [ ]:
anova(houseprices.lm)

In [ ]:
par(mfrow=c(2,2))

plot(houseprices.lm, which=1:4)
# By default, plots 1:3 and 5 [which=c(1:3,5)] are given
par(mfrow=c(1,1))

## 2. Plot confidence intervals

In [ ]:
# Confidence interval calculations
SEb = summary(houseprices.lm)$coefficients[2, 2]
coef(houseprices.lm)[2] + qt(c(0.025, 0.975), 8)*SEb

In [ ]:
# Obtain fitted values and standard errors (SE, then SE.OBS)
fit.with.se = predict(houseprices.lm, se.fit=TRUE)

In [ ]:
# SE : Standard error
fit.with.se$se.fit

In [ ]:
# SE.OBS : Precision of predicting an observation
sqrt(fit.with.se$se.fit**2+fit.with.se$residual.scale**2)

In [ ]:
# Plot sale price vs floor area, with 95\% pointwise bounds for the fitted line
plot(sale.price ~ area, data=houseprices,
     xlab = "Floor area [m^2]",
     ylab = "Sale price [1000 USD]", pch = 16)

abline(houseprices.lm, lty = 2)

xy = data.frame(area = pretty(houseprices$area, 20))
yhat = predict(houseprices.lm, newdata = xy, interval="confidence")
ci = data.frame(lower=yhat[, "lwr"], upper=yhat[, "upr"])
lines(xy$area, ci$lower, lty = 2, lwd=2, col="grey")
lines(xy$area, ci$upper, lty = 2, lwd=2, col="grey")

## 3. K-fold CV on `lm()` calculations

<img src="./images/kfoldcv.png" width="1200px">

In [ ]:
CVout = CVlm(houseprices, houseprices.lm, m=3, plotit=TRUE)

### Calculate error of K-fold model

The K-Fold error is even bigger, than in the naive case! Using bigger K values this can be even reversed. Eg. K=10 is a good choice for a bigger dataset.

In [ ]:
# From model
summary(houseprices.lm)$sigma**2

# From K-fold CV
(8684 + 14083 + 26421) / 15

## 4. Bootstrapping

In [ ]:
library(boot)

### a.) Compute standard errors of slope

In [ ]:
houseprices.fn = function (houseprices, index){

    # Randomly resample data
    house.resample = houseprices[index, ]
    # Fit linear model on the new dataset
    house.lm = lm(sale.price ~ area, data=house.resample)
    # Return the slope coefficient of the fit
    coef(house.lm)[2]
}

In [ ]:
houseprices.boot = boot(houseprices, R=999, statistic=houseprices.fn)

In [ ]:
houseprices.boot

### b.) Compute confidence intervals

In [ ]:
housepred.fn = function(houseprices, index){

    # Randomly resample data
    house.resample = houseprices[index, ]
    # Fit linear model on the new dataset
    house.lm = lm(sale.price ~ area, data=house.resample)
    # The last one is the bootstrapped statistics
    predict(house.lm, newdata=data.frame(area=1200))
}

In [ ]:
housepred.boot = boot(houseprices, R=999, statistic=housepred.fn)

In [ ]:
# 95% CI for predicted price of 1200 square foot house
boot.ci(housepred.boot, type="perc")

### c.) Bootstrap estimates of prediction errors of house prices

In [ ]:
houseprices2.fn = function (houseprices, index)
{
    house.resample = houseprices[index, ]
    house.lm = lm(sale.price ~ area, data=house.resample)
    houseprices$sale.price - predict(house.lm, houseprices) # resampled prediction
                                                            # errors
}

In [ ]:
n = length(houseprices$area)
R = 200
houseprices2.boot = boot(houseprices, R=R, statistic=houseprices2.fn)
house.fac = factor(rep(1:n, rep(R, n)))

In [ ]:
plot(house.fac, as.vector(houseprices2.boot$t),
     ylab="Prediction Errors",
     xlab="House")
## Ratios of bootstrap to model-based standard errors
bootse = apply(houseprices2.boot$t, 2, sd)
usualse = predict.lm(houseprices.lm, se.fit=TRUE)$se.fit
plot(bootse/usualse,
     ylab="Ratio of Bootstrap SE’s to Model-Based SE’s",
     xlab="House", pch=16)
abline(1, 0)